<a href="https://colab.research.google.com/github/MarkoGonvil/cryp-intel/blob/main/Cryp_Data_Analytics_Tool_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment Set up

##### Libraries Set up

In [ ]:
#Libraries Import
import requests
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

### Setting up Functions definition

##### Function to fetch api info from cryptocompare.com

In [ ]:
#Crypto Comparte API Call for Hystory of Hourly Prices Update
def get_hourly_price_data(symbol, comparison_symbol, limit, aggregate, exchange=''):
  url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}'\
          .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
  if exchange:
      url += '&e={}'.format(exchange)
  page = requests.get(url)
  data = page.json()['Data']
  df = pd.DataFrame(data)
  df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
  
  #Print data points and max length of day plotted
  print('Max Length = %s' % len(df))
  print('Max Time = %s' % (df.timestamp.max()-df.timestamp.min()))
  
  #Volume get the delta of volume
  df["volume"] = abs(df.volumeto - df.volumefrom)


  return df

##### Function to get the MACD numbers

In [ ]:
def get_macd(df, fast_trend, slow_trend, macd_periods):
  
  #Create the dimensions for macd values calculation
  df["exp1_macd"] = df.close.ewm(span= fast_trend, adjust=False).mean()
  df["exp2_macd"] = df.close.ewm(span= slow_trend, adjust=False).mean()
  df["macd"] = df.exp1_macd - df.exp2_macd
  df["macd_signal"] = df.macd.ewm(span= macd_periods, adjust=False).mean()
  
  below_tag = 0
  above_tag = 0
  df["macd_crossing_tag"] = None
  df["macd_crossing_value"] = None
  df["macd_recom"] = None

  for i in range(len(df)):
    if df.at[i, "macd"] <= df.at[i, "macd_signal"]:
      below_tag = 1
  
    elif df.at[i, "macd"] > df.at[i, "macd_signal"]:
      above_tag = 1
    
    if below_tag == above_tag:
      df.at[i, "macd_crossing_tag"] = 1
      df.at[i, "macd_crossing_value"] = df.at[i, "macd"]
      if df.at[i-1, "macd"] < df.at[i-1, "macd_signal"]:
        df.at[i, "macd_recom"] = "Buy"
      else:
        df.at[i, "macd_recom"] = "Sell"
  
      below_tag = 0
      above_tag = 0

  return df

##### Function for plotting MACD

In [ ]:
def plot_macd(df, crypto_currency, forex_pair):
  #Plot MACD
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df_mod_price_data.timestamp, y=df_mod_price_data.macd,
                      mode='lines',
                      name='MACD'))
  fig.add_trace(go.Scatter(x=df_mod_price_data.timestamp, y=df_mod_price_data.macd_signal,
                      mode='lines',
                      name='MACD Signal'))
  fig.add_trace(go.Scatter(x=df_mod_price_data.timestamp, y=df_mod_price_data["macd_crossing_value"],
                      mode='markers',
                      name='Cross'))
  
  fig.update_layout(template="plotly_dark", title="MACD Signals "+crypto_currency +"/"+forex_pair)

  return fig

##### Function to get the ADX numbers

In [ ]:
def get_adx(df, lookback, adx_break_point):
    plus_dm = df.high.diff()
    minus_dm = df.low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(df.high - df.low)
    tr2 = pd.DataFrame(abs(df.high - df.close.shift(1)))
    tr3 = pd.DataFrame(abs(df.low - df.close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    df["plus_D"] = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    df["minus_D"] = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(df.plus_D - df.minus_D) / abs(df.plus_D + df.minus_D)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    df["ADX"] = adx.ewm(alpha = 1/lookback).mean()
    df["ADX_break"] = adx_break_point

    df["ADX_Comment_Direction"] = np.where(df["plus_D"] >= df["minus_D"], "Bullish", "Bearish")
    df["ADX_Comment_Strenght"] = np.where(df["ADX"] > df["ADX_break"], "Strong_mov", "Weak_mov")

    return df

##### Function for plotting ADX

In [ ]:
def plot_adx(df, crypto_currency, forex_pair):
  #Plot ADX
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df.timestamp, y=df.ADX,
                      mode='lines',
                      line=dict(color='blue', width=4),
                      name='ADX'))
  fig.add_trace(go.Scatter(x=df.timestamp, y=df.plus_D,
                      mode='lines',
                      line=dict(color='green', width=1),
                      name='D+'))
  fig.add_trace(go.Scatter(x=df.timestamp, y=df.minus_D,
                      mode='lines',
                      #fill= 'tonexty',
                      #fillcolor= np.where(df["plus_D"]>= df["minus_D"], 'green', 'red'),
                      line=dict(color='red', width=1),
                      name='D-'))
  fig.add_trace(go.Scatter(x=df.timestamp, y=df.ADX_break,
                      mode='lines',
                      line=dict(color='white', width=1, dash= 'dash'),
                      name='D-'))
  
 
  fig.update_layout(template="plotly_dark", title="ADX "+crypto_currency +"/"+forex_pair)
  
  return fig

##### Function to get the Moving Average numbers for price supports

In [ ]:
#Moving Average for trends

def get_ma_supports(df, slow_ma, fast_ma, funct):
  df['MA_slow'] = df.close.ewm(span= slow_ma, adjust=False).mean()
  df['MA_fast'] = df.close.ewm(span= fast_ma, adjust=False).mean()
  return df

#####Function to get the Fibonacci Follower MA

In [ ]:
def get_fibonacci_ma_follower(df, long_ma, followers):
  df['MA_Fibonacci_Long'] = df.close.ewm(span= long_ma, adjust=False).mean()
  df['MA_Fibonacci_GoldenRatio'] = df.close.ewm(span= long_ma, adjust=False).mean() * 1.6
  df['MA_Fibonacci_2'] = df.close.ewm(span= long_ma, adjust=False).mean() * 2
  df['MA_Fibonacci_3'] = df.close.ewm(span= long_ma, adjust=False).mean() * 3
  df['MA_Fibonacci_5'] = df.close.ewm(span= long_ma, adjust=False).mean() * 5
  df['MA_Fibonacci_8'] = df.close.ewm(span= long_ma, adjust=False).mean() * 8
  df['MA_Fibonacci_13'] = df.close.ewm(span= long_ma, adjust=False).mean() * 13
  df['MA_Fibonacci_21'] = df.close.ewm(span= long_ma, adjust=False).mean() * 21
  return df

##### Function for plotting General price Analysis

In [ ]:
def plot_price_vol_analysis(df, price_pace=0.25, return_raw=False):
    """
    create volume profile
    :param df: time-indexed HOLCV bar or time-indexed P-V tick
    :param price_pace: price bucket, default 5 cents
    :param return_raw: return raw data or figure
    :return: raw data or figure obj
    """
 
    cmin = min(df.close)
    cmax = max(df.close)
    cmin_int = int(cmin / price_pace) * price_pace  # int(0.9) = 0
    cmax_int = int(cmax / price_pace) * price_pace
    if cmax_int < cmax:
        cmax_int += price_pace
    cmax_int += price_pace  # right bracket is not included in arrange
 
    price_buckets = np.arange(cmin_int, cmax_int, price_pace)
    price_coors = pd.Series(price_buckets).rolling(2).mean().dropna()
    vol_bars = np.histogram(df.close, bins=price_buckets, weights=df.volume)[0]
 
    if return_raw:
        return (price_coors.values, vol_bars)
 
    price_plot = go.Candlestick(
        x=df.timestamp,
        open=df.open,
        high=df.high,
        low=df.low,
        close=df.close,
        xaxis='x',
        yaxis='y2',
        visible=True,
        showlegend=False
    )
 
    vol_plot_x = go.Bar(
        x=df.timestamp,
        y=df.volume,
        yaxis='y',
        name='Volume',
        showlegend=False
    )
 
    vol_plot_y = go.Bar(
        x=vol_bars,
        y=price_coors.values,
        orientation='h',
        xaxis='x2',
        yaxis='y3',
        visible=True,
        showlegend=False,
        marker_color='white',
        opacity=0.5
    )
 
    ma_slow_plot = go.Scatter( 
        x=df.timestamp, 
        y=df.MA_slow,
        mode='lines',
        yaxis = 'y2',
        name= 'MA_Slow',
        marker_color = 'orange'
    )
 
    ma_fast_plot = go.Scatter(
        x=df.timestamp, 
        y=df.MA_fast,
        mode='lines',
        yaxis= 'y2',
        name= 'MA_Fast',
        marker_color = 'yellow'
    )
 
    macd_signal = go.Scatter(
        x=df.timestamp,
        y=np.where(df.macd_crossing_tag == 1, df.close, None),
        mode='markers',
        yaxis='y2',
        name= 'MACD_Signal',
        marker_symbol= np.where(df.macd_recom == "Buy", 'triangle-up', "triangle-down"),
        marker_size= 13,
        marker_color = np.where(df.macd_recom == "Buy", "cyan", "magenta")
    )
 
    adx_signal = go.Bar(
        x=df.timestamp,
        y=np.where(df.ADX_Comment_Strenght == "Strong_mov", max(df.close), None),
        yaxis='y2',
        marker_color=np.where(df.ADX_Comment_Direction == "Bullish", "green", np.where(df.ADX_Comment_Direction == "Bearish", "red", None)),
        opacity= 0.3, #np.where(df.ADX < 35, 0.1, np.where(df.ADX < 45, 0.2, np.where(df.ADX < 55, 0.3, np.where(df.ADX < 65, 0.4, np.where(df.ADX < 0.75, 0.5, 0.6))))),
        name= 'ADX_Signal'
    )

    ma_fibonacci_follower = go.Scatter(
        x=df.timestamp, 
        y= df.MA_Fibonacci_Long,
        mode='lines',
        yaxis= 'y2',
        name= 'MA_Fibo_Long',
        line = dict(color='royalblue', width=3, dash='dash')
    )

    ma_fibonacci_follower_GoldenRatio = go.Scatter(
        x=df.timestamp, 
        y= df.MA_Fibonacci_GoldenRatio,
        mode='lines',
        yaxis= 'y2',
        name= 'MA_Fibo_GR',
        line = dict(color='royalblue', width=3, dash='dash')
    )

    ma_fibonacci_follower_2 = go.Scatter(
        x=df.timestamp, 
        y= df.MA_Fibonacci_2,
        mode='lines',
        yaxis= 'y2',
        name= 'MA_Fibo_2',
        line = dict(color='royalblue', width=3, dash='dash')
    )

    ma_fibonacci_follower_3 = go.Scatter(
        x=df.timestamp, 
        y= df.MA_Fibonacci_3,
        mode='lines',
        yaxis= 'y2',
        name= 'MA_Fibo_3',
        line = dict(color='royalblue', width=3, dash='dash')
    )

    ma_fibonacci_follower_5 = go.Scatter(
        x=df.timestamp, 
        y= df.MA_Fibonacci_5,
        mode='lines',
        yaxis= 'y2',
        name= 'MA_Fibo_5',
        line = dict(color='royalblue', width=3, dash='dash')
    )

    ma_fibonacci_follower_8 = go.Scatter(
        x=df.timestamp, 
        y= df.MA_Fibonacci_8,
        mode='lines',
        yaxis= 'y2',
        name= 'MA_Fibo_8',
        line = dict(color='royalblue', width=3, dash='dash')
    )
 
    low = cmin_int
    high = cmax_int
    layout = go.Layout(
        title=go.layout.Title(text="Volume Profile " + crypto_currency + "/" + forex_pair),
        xaxis=go.layout.XAxis(
            side="bottom",
            title="Date",
            rangeslider=go.layout.xaxis.Rangeslider(visible=False)
        ),
        yaxis=go.layout.YAxis(
            side="right",
            title='Volume',
            showticklabels=False,
            domain=[0, 0.2]
        ),
        yaxis2=go.layout.YAxis(
            side="right",
            title='Price',
            range=[low, high],
            domain=[0.2, 1.0]
        ),
        xaxis2=go.layout.XAxis(
            side="top",
            showgrid=False,
            # volume bar on the right
            # unfortunately reversed is an auto-range
            # one solution is to add an invisible bar.
            # https://community.plotly.com/t/reversed-axis-with-range-specified/3806
            # autorange='reversed',
            ticks='',
            showticklabels=False,
            range=[0, int(vol_bars.max() * 5)],
            overlaying="x"
        ),
        yaxis3=go.layout.YAxis(
            side="left",
            range=[low, high],
            showticklabels=False,
            overlaying="y2"
        ),
    )
 
    fig = go.Figure(data=[price_plot, vol_plot_x, vol_plot_y, ma_slow_plot, ma_fast_plot, macd_signal, adx_signal, ma_fibonacci_follower, ma_fibonacci_follower_GoldenRatio,
                          ma_fibonacci_follower_2, ma_fibonacci_follower_3,
                          ma_fibonacci_follower_5, ma_fibonacci_follower_8], 
                    layout=layout)
    fig.update_layout(template="plotly_dark", title="Analysis "+crypto_currency +"/"+forex_pair)
 
    return fig

##### Function for strategy set up evaluation Profit and Loss

In [ ]:
def eval_pl_strategy(df):
  df_orders_sim = pd.DataFrame(index= range(0,1000), columns={"Order_Action", "Price", "Delta", "Profit_Loss"})
  
  j = 0
  for i in range(len(df)):
    if df.at[i, "macd_recom"] != None:
      df_orders_sim.at[j, "Order_Action"] = df.at[i, "macd_recom"]
      df_orders_sim.at[j, "Price"] = df.at[i, "close"]
      j = j +1
  
  for i in range(len(df_orders_sim)):
    if df_orders_sim.at[i, "Order_Action"] == "Sell":
      df_orders_sim.at[i, "Delta"] = (df_orders_sim.at[i, "Price"]*1.0188) - df_orders_sim.at[i-1, "Price"]


  df_orders_sim["Profit_Loss"] = np.where(df_orders_sim["Delta"].fillna(0) > 0, "Profit", np.where(df_orders_sim["Delta"].fillna(0) < 0, "Loss", None))


  print("Strategy Profit/Loss: " + str(sum(df_orders_sim.Delta.fillna(0))))
  print("Orders with Profit: " + str(sum(df_orders_sim.Profit_Loss.str.count("Profit").fillna(0))))
  print("Mean Profit: " + str(df_orders_sim.Profit_Loss.str.count("Profit").fillna(0).mean()))
  print("Orders with Loss: " + str(sum(df_orders_sim.Profit_Loss.str.count("Loss").fillna(0))))
  print("Mean Loss: " + str(df_orders_sim.Profit_Loss.str.count("Loss").fillna(0).mean()))

  return df_orders_sim


### Execute Technical Analysis

##### Getting and Processing Data for Crypto

In [ ]:
#Change variables to custom the function to import price data
crypto_currency = 'ADA'
forex_pair = 'USD'
limit_data = 200
time_delta = 4

#Create DataFrame for Prices and Volume
df_raw_price_data = get_hourly_price_data(crypto_currency, forex_pair, limit_data, time_delta)

#Create dataframe for Technical Analysis
df_mod_price_data = df_raw_price_data

#MACD Setup
fast_trend = 12
slow_trend = 26
macd_periods = 9

df_mod_price_data = get_macd(df_mod_price_data, fast_trend, slow_trend, macd_periods)

#ADX Set Up
lookback = 14
adx_boundary = 23

df_mod_price_data = get_adx(df_mod_price_data, lookback, adx_boundary)

#Moving Average Price Supports Set up
slow_ma = 54
fast_ma = 21
agg_function = "mean"

df_mod_price_data = get_ma_supports(df_mod_price_data, slow_ma, fast_ma, agg_function)

#Fibonacci Followers
long_ma =42

df_mod_price_data = get_fibonacci_ma_follower(df_mod_price_data, long_ma, 10)

Max Length = 201
Max Time = 33 days 08:00:00


##### Plot the technical analysis

In [ ]:
#Evaluate the Strategy Set up for Profit&Loss
#eval_pl_strategy(df_mod_price_data)

#Plot
fig = plot_price_vol_analysis(df_mod_price_data, price_pace=0.05)
fig.show()
fig = plot_macd(df_mod_price_data, crypto_currency, forex_pair)
fig.show()
fig= plot_adx(df_mod_price_data, crypto_currency, forex_pair)
fig.show()

##### Evaluating best Strategy

In [ ]:
#Change variables to custom the function to import price data
crypto_currency = 'ADA'
forex_pair = 'USD'
limit_data = 2000
time_delta = 4

#Create DataFrame for Prices and Volume
df_raw_price_data = get_hourly_price_data(crypto_currency, forex_pair, limit_data, time_delta)

#Create dataframe for Technical Analysis
df_mod_price_data = df_raw_price_data

#MACD Setup

df_strategy_eval_macd = pd.DataFrame(index= range(0,150000), columns={"Iteration", "Fast_trend", "Slow_trend", "MACD_Lookback", "Profit_Loss", "Orders_w_Profit", "Orders_w_Loss", "Mean_Profit", "Mean_Loss"})

strat_counter = 0

for i in range(1, 20):
  fast_trend = i

  for j in range(1, 20):
    slow_trend = j

    for k in range(1, 20):
      macd_periods = k

      print("Fast Trend: ", fast_trend)
      print("Slow Trend: ", slow_trend)
      print("MACD Lookback: ", macd_periods)
      print("Strat_Counter: ", strat_counter)

      df_test = get_macd(df_mod_price_data, fast_trend, slow_trend, macd_periods)
      df_out = eval_pl_strategy(df_test)

      df_strategy_eval_macd["Iteration"][strat_counter] = strat_counter
      df_strategy_eval_macd["Fast_trend"][strat_counter] = fast_trend
      df_strategy_eval_macd["Slow_trend"][strat_counter] = slow_trend
      df_strategy_eval_macd["MACD_Lookback"][strat_counter] = macd_periods
      df_strategy_eval_macd["Profit_Loss"][strat_counter] = sum(df_out.Delta.fillna(0))
      df_strategy_eval_macd["Orders_w_Profit"][strat_counter] = sum(df_out.Profit_Loss.str.count("Profit").fillna(0))
      df_strategy_eval_macd["Mean_Profit"][strat_counter] = df_out.Profit_Loss.str.count("Profit").fillna(0).mean()
      df_strategy_eval_macd["Orders_w_Loss"][strat_counter] = sum(df_out.Profit_Loss.str.count("Loss").fillna(0))
      df_strategy_eval_macd["Mean_Loss"][strat_counter] = df_out.Profit_Loss.str.count("Loss").fillna(0).mean()

      strat_counter = strat_counter + 1

max(df_strategy_eval_macd["Fast_trend"])
max(df_strategy_eval_macd["Slow_trend"])
max(df_strategy_eval_macd["MACD_Lookback"])



Max Length = 501
Max Time = 83 days 08:00:00
Fast Trend:  1
Slow Trend:  1
MACD Lookback:  1
Strat_Counter:  0
Strategy Profit/Loss: 0
Orders with Profit: 0
Mean Profit: 0.0
Orders with Loss: 0
Mean Loss: 0.0
Fast Trend:  1
Slow Trend:  1
MACD Lookback:  2
Strat_Counter:  1
Strategy Profit/Loss: 0
Orders with Profit: 0
Mean Profit: 0.0
Orders with Loss: 0
Mean Loss: 0.0
Fast Trend:  1
Slow Trend:  1
MACD Lookback:  3
Strat_Counter:  2
Strategy Profit/Loss: 0
Orders with Profit: 0
Mean Profit: 0.0
Orders with Loss: 0
Mean Loss: 0.0
Fast Trend:  1
Slow Trend:  1
MACD Lookback:  4
Strat_Counter:  3
Strategy Profit/Loss: 0
Orders with Profit: 0
Mean Profit: 0.0
Orders with Loss: 0
Mean Loss: 0.0
Fast Trend:  1
Slow Trend:  1
MACD Lookback:  5
Strat_Counter:  4
Strategy Profit/Loss: 0
Orders with Profit: 0
Mean Profit: 0.0
Orders with Loss: 0
Mean Loss: 0.0
Fast Trend:  1
Slow Trend:  1
MACD Lookback:  6
Strat_Counter:  5
Strategy Profit/Loss: 0
Orders with Profit: 0
Mean Profit: 0.0
Orders

KeyError: ignored

In [ ]:
x_axi = df_strategy_eval_macd["Fast_trend"]
y_axi = df_strategy_eval_macd["Slow_trend"]
z_axi = df_strategy_eval_macd["MACD_Lookback"]

fig = px.scatter_3d(df_strategy_eval_macd, x= 'Fast_trend', y= 'Profit_Loss', z= 'MACD_Lookback')
fig.show()

df_orders_sim.at[i, "Delta"] = (df_orders_sim.at[i, "Price"]*1.0188) - df_orders_sim.at[i-1, "Price"]

NameError: ignored